In [ ]:
!pip install geopandas

In [ ]:
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
import pandas as pd
import seaborn as sns

In [ ]:
# Lê a base de dados

import geopandas as gpd
import urllib.request

# URL do arquivo GeoJSON no Dropbox
url = "https://www.dropbox.com/s/ii83lo2nzc0vuun/bairros.geojson?dl=1"

# Nome do arquivo local para salvar o GeoJSON
arquivo_local = "bairros.geojson"

# Baixa o arquivo GeoJSON do Dropbox
urllib.request.urlretrieve(url, arquivo_local)

# Carrega o arquivo GeoJSON com o geopandas
df_bairros = gpd.read_file(arquivo_local)

df_bairros.head(2)

df_bairros["lon"] = df_bairros["geometry"].centroid.x
df_bairros["lat"] = df_bairros["geometry"].centroid.y


<ipython-input-3-10694c6b5367>:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_bairros["lon"] = df_bairros["geometry"].centroid.x
<ipython-input-3-10694c6b5367>:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_bairros["lat"] = df_bairros["geometry"].centroid.y


In [ ]:
# Lista de bairros
bairros_f = ['Jacintinho', 'Chã da Jaqueira', 'Bom Parto', 'Vergel do Lago', 'Ponta Grossa', 'Trapiche da Barra',
           'Jatiúca', 'Levada', 'Clima Bom', 'Prado', 'Feitosa', 'Ouro Preto', 'Poço', 'Pinheiro', 'Ponta Verde',
           'Canaã', 'Chã de Bebedouro', 'Tabuleiro do Martins', 'Santo Amaro', 'Santa Lúcia', 'Barro Duro', 'Farol',
           'Cruz das Almas', 'Mangabeiras', 'Petrópolis', 'Mutange', 'Pitanguinha', 'Bebedouro', 'Santa Amélia',
           'Gruta de Lourdes', 'São Jorge', 'Benedito Bentes', 'Cidade Universitária', 'Serraria', 'Santos Dumont',
           'Antares', 'Rio Novo', 'Jaraguá', 'Fernão Velho', 'Ponta da Terra', 'Jardim Petrópolis', 'Pontal da Barra',
           'Centro', 'Jacarecica', 'Garça Torta', 'Pescaria', 'Pajuçara', 'Riacho Doce', 'Guaxumã', 'Ipioca']

In [ ]:
# Filtra do dataframe apenas os bairros de Maceió

url = 'https://www.dropbox.com/s/ii83lo2nzc0vuun/bairros.geojson?dl=1'
df = pd.read_json(url)


df_bairros = df_bairros[df_bairros['Bairro'].isin(bairros_f)]

In [ ]:
# Lê a base de dados de densidade demográfica

url = 'https://docs.google.com/spreadsheets/d/1kAsPjy2EgTOv5wKkDFHhBsl2OQpHp5KwlZ95sjC59Dw/export?format=csv'

dados_dens = pd.read_csv(url, index_col=0, decimal=',', usecols=['Bairro', 'Densidade Demográfica'])


In [ ]:
# Cria um novo dataframe com a densidade demográfica e os bairros

df_mcz = pd.merge(df_bairros, dados_dens, on = "Bairro")


In [ ]:
# Exclui colunas desnecessárias no dataframe

excluir = ['População Feminina', 'População 2010', 'População Masculina']
df = pd.DataFrame(df_mcz)
for i in excluir:
    df_mcz = df.drop(i, axis=1)
    df = df_mcz


In [ ]:
# Tratamento dos dados do dataframe

coordenadas = df_mcz[['lat','lon','Densidade Demográfica']]

def convert(value: str | float) -> float:
  if type(value) == str:
    return float(value.replace(".", "").replace(",", "."))

  return value

coordenadas = [list(map(convert, coord)) for coord in coordenadas.values.tolist()]

In [ ]:
# Cria o mapa e o HeatMap

mapa_mcz = folium.Map(
    location=[-9.6037857,-35.7013138],
    zoom_start=12)

mapa_mcz = mapa_mcz.add_child(HeatMap(coordenadas))

In [ ]:
mapa_mcz